<br>

# Introdução

- https://www.tjsp.jus.br
- https://www.tjsp.jus.br/QuemSomos/QuemSomos/RegioesAdministrativasJudiciarias
- https://www.tjsp.jus.br/ListaTelefonica


In [ ]:
#!pip3 install webdriver_manager
#!pip3 install traquitanas


In [ ]:
import pandas as pd
from driver import driver
from paths import *
from selenium.webdriver.common.by import By

In [ ]:
from traquitanas import net

net

In [ ]:
URL = 'https://www.tjsp.jus.br/QuemSomos/QuemSomos/RegioesAdministrativasJudiciarias'
driver.get(URL)


In [ ]:
driver.window_handles


In [ ]:
driver.switch_to.window(driver.window_handles[0])


In [ ]:
# Localiza o Termo "Juízes Diretores"
driver.find_element(
    By.XPATH, '//*[text()="Juízes Diretores"]').get_attribute('innerHTML')

# Localiza <div> irmã dos "Juízes Diretores"
juiz_xpath = driver.find_element(
    By.XPATH, '//*[text()="Juízes Diretores"]//..')
rajs_xpath = juiz_xpath.find_element(By.XPATH, './following-sibling::div')


In [ ]:
rajs_xpaths = rajs_xpath.find_elements(
    By.XPATH, './/*[contains(@style, "background")]')
rajs_xpaths


In [ ]:
df = pd.DataFrame()

for i in rajs_xpaths:
    # Xpath
    raj_xpath = i.find_element(By.XPATH, './/p[@style="font-weight: bold"]')
    diretor_xpath = raj_xpath.find_element(By.XPATH, "./following-sibling::p")
    email_xpath = diretor_xpath.find_element(
        By.XPATH, "./following-sibling::p")

    # Print
    # print(raj_xpath.text)
    # print(diretor_xpath.text)
    # print(email_xpath.text)
    dict_circuncricao = None
    dict_circuncricao = {
        'raj': raj_xpath.text,
        'juiz_diretor': diretor_xpath.text,
        'email': email_xpath.text,        
    }    

    # Para cada 'ul' em todas as 'uls'
    for ul in i.find_elements(By.XPATH, "./following-sibling::ul"):

        # Procura todas as 'li' em cada 'ul
        for li in ul.find_elements(By.XPATH, './/li[@class="list-group-item"]'):
            # Atualiza Dicionário
            dict_circuncricao.update({'comarca_cirscunscricao': li.text})

            # Convert para DataFrame
            df_dict = pd.DataFrame([dict_circuncricao])

            # Concatena
            df = pd.concat([df, df_dict], ignore_index=True)

# Results
df

In [ ]:
df['raj'] = df['raj'].str.replace('–', '-')
df[['raj_id','raj_nome']] = df['raj'].str.split(' - ',expand=True)
df[['comarca','cirscunscricao_judiciaria']] = df['comarca_cirscunscricao'].str.split(' - ',expand=True)
df.head()

In [ ]:
list(df.columns)

In [ ]:
df = df[[
    'raj',
    'raj_id',
    'raj_nome',

    'juiz_diretor',
    'email',
    'comarca_cirscunscricao',
    'comarca',
    'cirscunscricao_judiciaria'
    ]]

df.to_csv(
    output_path / 'raj.csv',
    index=False,
)